In [10]:
!/home/renku/work/exiftool/exiftool -a -G1 -s -XMP:all -XMP-xmpMM:all "/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/04_Entwurf/060625_FinalDesign/02_Texturen/v1_8_r3_cam1_simplified_inv.jpg"

[XMP-x]         XMPToolkit                      : 3.1.1-112
[XMP-xmpMM]     DocumentID                      : uuid:11B81C2FCA03DB11958680BA79BAC3A6
[XMP-xmpMM]     InstanceID                      : uuid:12B81C2FCA03DB11958680BA79BAC3A6
[XMP-xmpMM]     DerivedFromInstanceID           : uuid:0E1E7741BA03DB11958680BA79BAC3A6
[XMP-xmpMM]     DerivedFromDocumentID           : uuid:0D1E7741BA03DB11958680BA79BAC3A6
[XMP-xmp]       CreateDate                      : 2006:06:24 23:41:53+02:00
[XMP-xmp]       ModifyDate                      : 2006:06:24 23:41:53+02:00
[XMP-xmp]       MetadataDate                    : 2006:06:24 23:41:53+02:00
[XMP-xmp]       CreatorTool                     : Adobe Photoshop CS2 Windows
[XMP-dc]        Format                          : image/jpeg
[XMP-photoshop] ColorMode                       : RGB
[XMP-photoshop] History                         : 
[XMP-tiff]      Orientation                     : Horizontal (normal)
[XMP-tiff]      XResolution                   

In [12]:
!/home/renku/work/exiftool/exiftool -json -a -G1 -s -struct -XMP:all -XMP-xmpMM:all "/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/04_Entwurf/060625_FinalDesign/02_Texturen/v1_8_r3_cam1_simplified_inv.jpg"

[{
  "SourceFile": "/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/04_Entwurf/060625_FinalDesign/02_Texturen/v1_8_r3_cam1_simplified_inv.jpg",
  "XMP-x:XMPToolkit": "3.1.1-112",
  "XMP-xmpMM:DocumentID": "uuid:11B81C2FCA03DB11958680BA79BAC3A6",
  "XMP-xmpMM:InstanceID": "uuid:12B81C2FCA03DB11958680BA79BAC3A6",
  "XMP-xmpMM:DerivedFrom": {
    "DocumentID": "uuid:0D1E7741BA03DB11958680BA79BAC3A6",
    "InstanceID": "uuid:0E1E7741BA03DB11958680BA79BAC3A6"
  },
  "XMP-xmp:CreateDate": "2006:06:24 23:41:53+02:00",
  "XMP-xmp:ModifyDate": "2006:06:24 23:41:53+02:00",
  "XMP-xmp:MetadataDate": "2006:06:24 23:41:53+02:00",
  "XMP-xmp:CreatorTool": "Adobe Photoshop CS2 Windows",
  "XMP-dc:Format": "image/jpeg",
  "XMP-photoshop:ColorMode": "RGB",
  "XMP-photoshop:History": "",
  "XMP-tiff:Orientation": "Horizontal (normal)",
  "XMP-tiff:XResolution": 72,
  "XMP-tiff:YResolution": 72,
  "XMP-tiff:ResolutionUnit":

In [5]:
!pip install pandas networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.3 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import os
import json
import subprocess
from pathlib import Path
import pandas as pd
import networkx as nx
from IPython.display import display, Markdown

# ------------------------------------------------------------
# 1) KONFIGURATION – GANZER NEXTCLOUD-ORDNER
# ------------------------------------------------------------
ROOT = Path("/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/")
EXTENSIONS = {".jpg", ".jpeg", ".tif", ".tiff", ".png", ".psd", ".psb"}

display(Markdown(f"### 📂 Scanne folgendes Wurzelverzeichnis:\n`{ROOT}`"))

# ------------------------------------------------------------
# 2) EXIFTOOL: ALLE XMP-SEGMENTE AUSLESEN (XMP + APP13/8BIM)
# ------------------------------------------------------------
def extract_xmpmm(path):
    """
    Extrahiert ALLE XMP-MM Felder aus Standard-XMP + Photoshop APP13/8BIM.
    """
    cmd = [
        "/home/renku/work/exiftool/exiftool",   # Pfad zu deinem ExifTool
        "-a",                          # alle doppelten Tags anzeigen
        "-G1",                         # zeigt Segment (XMP, APP1, APP13, etc.)
        "-s",                          # kurze Tag-Namen
        "-struct",                     # strukturierte Ausgabe
        "-XMP:all",                    # ALLE XMP-Namespace-Daten
        "-XMP-xmpMM:all",              # alle Media-Management-Felder
        "-json",
        str(path)
    ]
    
    try:
        out = subprocess.run(cmd, capture_output=True, text=True)
        data = json.loads(out.stdout)[0]
    except:
        return None
    
    # Mögliche Feld-Namen (wegen unterschiedlicher XMP-Segmente)
    keys = data.keys()

    def get(*names):
        for n in names:
            if n in data:
                return data[n]
        return None

    return {
        "file": str(path),
        "documentID": get("XMP-xmpMM-DocumentID", "DocumentID"),
        "instanceID": get("XMP-xmpMM-InstanceID", "InstanceID"),
        "parentDocumentID": get("XMP-xmpMM-DerivedFromDocumentID"),
        "parentInstanceID": get("XMP-xmpMM-DerivedFromInstanceID"),
    }

# ------------------------------------------------------------
# 3) SCAN DES GESAMTEN VERZEICHNISSES ÜBER WEBDAV
# ------------------------------------------------------------
display(Markdown("### 🔍 Durchsuche Dateien..."))

records = []
count = 0

for root, dirs, files in os.walk(ROOT):
    for f in files:
        p = Path(root) / f
        if p.suffix.lower() in EXTENSIONS:
            rec = extract_xmpmm(p)
            if rec:
                records.append(rec)
            count += 1

display(Markdown(f"### ✔️ Insgesamt gescannt: **{count} Dateien**"))
display(Markdown(f"### ✔️ Dateien mit XMP-MM-Genealogie: **{sum(1 for r in records if r['documentID'])}**"))

df = pd.DataFrame(records)
display(df.head(20))

# ------------------------------------------------------------
# 4) GENEALOGIE GRAPH
# ------------------------------------------------------------
G = nx.DiGraph()

for _, row in df.iterrows():
    doc = row["documentID"]
    parent = row["parentDocumentID"]

    if doc:
        G.add_node(doc)
    if parent:
        G.add_node(parent)
        G.add_edge(parent, doc)

display(Markdown("### 📌 Graph-Statistik"))
display({"nodes": G.number_of_nodes(), "edges": G.number_of_edges()})

# ------------------------------------------------------------
# 5) EXPORT GRAPH
# ------------------------------------------------------------
graph_json = nx.readwrite.json_graph.node_link_data(G)

with open("genealogy_graph_xmp_full.json", "w") as f:
    json.dump(graph_json, f, indent=2)

display(Markdown("### 📁 Genealogie-Graph gespeichert als `genealogy_graph_xmp_full.json`"))

# ------------------------------------------------------------
# 6) ALLE HERKUNFTS-KETTEN AUSGEBEN
# ------------------------------------------------------------
def print_chains(G):
    display(Markdown("## 🧬 Gefundene Herkunftsketten"))
    roots = [n for n in G.nodes if G.in_degree(n) == 0]
    for r in roots:
        chain = list(nx.dfs_preorder_nodes(G, source=r))
        print(" → ".join(chain))

print_chains(G)


### 📂 Scanne folgendes Wurzelverzeichnis:
`/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein`

### 🔍 Durchsuche Dateien...

### ✔️ Insgesamt gescannt: **12338 Dateien**

### ✔️ Dateien mit XMP-MM-Genealogie: **0**

,file,documentID,instanceID,parentDocumentID,parentInstanceID
0,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
1,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
2,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
3,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
4,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
5,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
6,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
7,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
8,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None
9,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,None,None,None,None


### 📌 Graph-Statistik

{'nodes': 0, 'edges': 0}

/home/renku/work/.venv/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


### 📁 Genealogie-Graph gespeichert als `genealogy_graph_xmp_full.json`

## 🧬 Gefundene Herkunftsketten

In [9]:
import time, itertools, os
from pathlib import Path
import subprocess

ROOT = Path("/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/")
EXT = {".jpg",".jpeg",".tif",".tiff",".png",".psd",".psb"}

def iter_files(root, limit=None):
    c = 0
    for r, d, files in os.walk(root):
        for f in files:
            if Path(f).suffix.lower() in EXT:
                yield Path(r)/f
                c += 1
                if limit and c >= limit:
                    return

def exif_call(p):
    return subprocess.run(
        ["/home/renku/work/exiftool/exiftool","-a","-G1","-s","-struct","-XMP:all","-XMP-xmpMM:all","-json",str(p)],
        capture_output=True, text=True
    )

# 1) Mini-Benchmark über 200 Dateien
N = 200
files = list(iter_files(ROOT, limit=N))
t0 = time.time()
for fp in files:
    exif_call(fp)
t1 = time.time()

per_file = (t1 - t0) / max(1, len(files))
print(f"Ø pro Datei: {per_file*1000:.1f} ms")

# 2) Gesamtdateien zählen
total = sum(1 for _ in iter_files(ROOT))
eta_sec = per_file * total
print(f"Gesamtdateien: {total} | ETA ≈ {eta_sec/60:.1f} Minuten")

Ø pro Datei: 340.0 ms
Gesamtdateien: 12338 | ETA ≈ 69.9 Minuten


In [13]:
import os, json, subprocess, threading, queue, sys, time
from pathlib import Path
import pandas as pd

# -----------------------------
# Konfiguration
# -----------------------------
EXIFTOOL = "/home/renku/work/exiftool/exiftool"  # dein ExifTool
ROOT = Path("/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/")
EXTENSIONS = {".jpg",".jpeg",".tif",".tiff",".png",".psd",".psb"}

# Globale Prozess-Referenz
_et_proc = None
_et_lock = threading.Lock()

def start_exiftool():
    global _et_proc
    if _et_proc is not None:
        return
    _et_proc = subprocess.Popen(
        [EXIFTOOL, "-stay_open", "True", "-@","-"], 
        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        text=True, bufsize=1
    )

def stop_exiftool():
    global _et_proc
    if _et_proc is None:
        return
    try:
        _et_proc.stdin.write("-stay_open\nFalse\n-execute\n")
        _et_proc.stdin.flush()
    except Exception:
        pass
    try:
        _et_proc.communicate(timeout=5)
    except Exception:
        _et_proc.kill()
    _et_proc = None

def et_request_json(files):
    """
    Schickt eine Anfrage (für eine oder mehrere Dateien) an den offenen ExifTool-Prozess
    und gibt die JSON-Liste zurück.
    """
    global _et_proc
    if _et_proc is None:
        start_exiftool()
    args = [
        "-a","-G1","-s","-struct",
        "-XMP:all","-XMP-xmpMM:all",
        "-json",
    ]
    # Schreibe Befehle + Dateien
    with _et_lock:
        for a in args:
            _et_proc.stdin.write(a+"\n")
        for f in files:
            _et_proc.stdin.write(str(f)+"\n")
        _et_proc.stdin.write("-execute\n")
        _et_proc.stdin.flush()

        # Antworte lesen: Wir wissen, dass -json nur EINEN JSON-Block je -execute schreibt.
        # Wir lesen so lange Zeilen, bis JSON „vermutlich“ vollständig ist (endet mit ']').
        out_lines = []
        while True:
            line = _et_proc.stdout.readline()
            if not line:
                break
            out_lines.append(line)
            # sehr einfache Heuristik: JSON-Array endet mit ']\n'
            if line.strip().endswith(']'):
                break

    out = "".join(out_lines).strip()
    if not out:
        return []

    try:
        data = json.loads(out)
        if isinstance(data, list):
            return data
        return [data]
    except json.JSONDecodeError:
        # Zur Diagnose ggf. stderr lesen (nicht zwingend in der Schleife auswerten)
        return []

def iter_files(root: Path, exts):
    for r, d, files in os.walk(root):
        for f in files:
            p = Path(r)/f
            if p.suffix.lower() in exts:
                yield p

def extract_from_json_item(item: dict):
    # JSON-Struktur wie in deinem Beispiel:
    # "XMP-xmpMM:DocumentID": "uuid:…",
    # "XMP-xmpMM:InstanceID": "uuid:…",
    # "XMP-xmpMM:DerivedFrom": {"DocumentID": "...", "InstanceID": "..."}
    doc = item.get("XMP-xmpMM:DocumentID") or item.get("DocumentID")
    ins = item.get("XMP-xmpMM:InstanceID") or item.get("InstanceID")
    parent_doc, parent_ins = None, None
    drv = item.get("XMP-xmpMM:DerivedFrom") or item.get("DerivedFrom")
    if isinstance(drv, dict):
        parent_doc = drv.get("DocumentID") or drv.get("documentID")
        parent_ins = drv.get("InstanceID") or drv.get("instanceID")
    return doc, ins, parent_doc, parent_ins

# -----------------------------
# Lauf: Dateien in Batches (Stay-Open bringt hier den Gewinn)
# -----------------------------
records = []
batch = []
BATCH_SIZE = 50  # 50–200: guter Kompromiss WebDAV <-> Antwortgröße

start_exiftool()
try:
    for fp in iter_files(ROOT, EXTENSIONS):
        batch.append(fp)
        if len(batch) >= BATCH_SIZE:
            data = et_request_json(batch)
            for item in data:
                # Quelle (immer enthalten)
                src = item.get("SourceFile")
                doc, ins, pdoc, pins = extract_from_json_item(item)
                records.append({
                    "file": src, 
                    "documentID": doc, 
                    "instanceID": ins, 
                    "parentDocumentID": pdoc, 
                    "parentInstanceID": pins
                })
            batch.clear()

    # Rest
    if batch:
        data = et_request_json(batch)
        for item in data:
            src = item.get("SourceFile")
            doc, ins, pdoc, pins = extract_from_json_item(item)
            records.append({
                "file": src, 
                "documentID": doc, 
                "instanceID": ins, 
                "parentDocumentID": pdoc, 
                "parentInstanceID": pins
            })
finally:
    stop_exiftool()

df = pd.DataFrame(records)
display(df.head(20))
print("Mit XMP‑MM:", df["documentID"].notna().sum(), "von", len(df))

KeyboardInterrupt: 

In [16]:
import os, json, subprocess, time
from pathlib import Path
import pandas as pd

# -----------------------------
# CONFIG
# -----------------------------
ROOT = Path("/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/")
EXT = {".jpg",".jpeg",".tif",".tiff",".png",".psd",".psb"}
EXIFTOOL = "/home/renku/work/exiftool/exiftool"
BATCH_SIZE = 50
MAX_TEST = 200  # nur Testlauf; für Vollscan entfernen/erhöhen
SENTINEL = "__END__"

print("📂 Testlauf (Stay-Open + Sentinel) für bis zu 200 Dateien:\n", ROOT)

# Dateien einsammeln (bis MAX_TEST)
files = []
for root, dirs, filelist in os.walk(ROOT):
    for f in filelist:
        p = Path(root)/f
        if p.suffix.lower() in EXT:
            files.append(str(p))
            if len(files) >= MAX_TEST:
                break
    if len(files) >= MAX_TEST:
        break

print(f"🔍 Teste jetzt {len(files)} Dateien...")

# -----------------------------
# ExifTool stay-open starten
# -----------------------------
et = subprocess.Popen(
    [EXIFTOOL, "-stay_open", "True", "-@", "-"],
    stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
    text=True, bufsize=1
)

def et_send_batch(batch_files):
    """
    Sendet EINEN Auftrag (Batch) an ExifTool:
      - schlanke Argumente (nur xmpMM)
      - JSON
      - Sentinel als EOT-Marker
    Liest stdout bis zum Sentinel, extrahiert den JSON-Block und liefert ihn als Liste zurück.
    """
    # SCHLANKE Argumente: nur xmpMM (schneller & weniger Output)
    args = ["-a", "-s", "-struct", "-XMP-xmpMM:all", "-json"]
    for a in args:
        et.stdin.write(a + "\n")
    for f in batch_files:
        et.stdin.write(f + "\n")
    # Sentinel vor -execute setzen
    et.stdin.write(f"-echo3\n{SENTINEL}\n")
    et.stdin.write("-execute\n")
    et.stdin.flush()

    # stdout lesen bis Sentinel gefunden
    lines = []
    t_start = time.time()
    while True:
        line = et.stdout.readline()
        if not line:
            break
        # Sentinel erreicht -> Ende der Antwort
        if line.strip() == SENTINEL:
            break
        lines.append(line)

    out = "".join(lines).strip()
    if not out:
        return []

    # JSON extrahieren (es kann vor/nach JSON noch andere Zeilen geben; wir finden den JSON-Array-Block)
    # Wir suchen den ersten '[' und das letzte ']'
    start = out.find('[')
    end   = out.rfind(']')
    if start == -1 or end == -1 or end < start:
        # Diagnose: zeige kurze Vorschau
        print("⚠️ Konnte JSON-Block nicht isolieren. Vorschau:", out[:400])
        return []

    json_str = out[start:end+1]
    try:
        data = json.loads(json_str)
        return data if isinstance(data, list) else [data]
    except json.JSONDecodeError as e:
        print("⚠️ JSONDecodeError:", e)
        print("Vorschau:", json_str[:400])
        return []

def et_close():
    try:
        et.stdin.write("-stay_open\nFalse\n-execute\n")
        et.stdin.flush()
        et.communicate(timeout=3)
    except Exception:
        et.kill()

# -----------------------------
# Batchweise abarbeiten
# -----------------------------
records = []
t0 = time.time()

try:
    for i in range(0, len(files), BATCH_SIZE):
        batch = files[i:i+BATCH_SIZE]
        data  = et_send_batch(batch)
        # pro Batch extrahieren
        for item in data:
            src = item.get("SourceFile")
            doc = item.get("XMP-xmpMM:DocumentID") or item.get("DocumentID")
            ins = item.get("XMP-xmpMM:InstanceID") or item.get("InstanceID")
            pdoc = pins = None
            drv = item.get("XMP-xmpMM:DerivedFrom") or item.get("DerivedFrom")
            if isinstance(drv, dict):
                pdoc = drv.get("DocumentID") or drv.get("documentID")
                pins = drv.get("InstanceID") or drv.get("instanceID")
            records.append({
                "file": src,
                "documentID": doc,
                "instanceID": ins,
                "parentDocumentID": pdoc,
                "parentInstanceID": pins
            })
finally:
    et_close()

t1 = time.time()
print(f"⏱️ Testlauf Dauer: {t1-t0:.2f} s")

df = pd.DataFrame(records)
print("\n🧬 Ergebnis (erste 20 Zeilen):")
display(df.head(20))
print("\n✔️ Dateien MIT XMP‑MM:", df["documentID"].notna().sum() if "documentID" in df.columns else 0, "von", len(df))

📂 Testlauf (Stay-Open + Sentinel) für bis zu 200 Dateien:
 /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein
🔍 Teste jetzt 200 Dateien...
⏱️ Testlauf Dauer: 48.58 s

🧬 Ergebnis (erste 20 Zeilen):


,file,documentID,instanceID,parentDocumentID,parentInstanceID
0,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,uuid:C74E838EEACFDA118C30CF8EA9BFE8B1,uuid:C84E838EEACFDA118C30CF8EA9BFE8B1,uuid:C54E838EEACFDA118C30CF8EA9BFE8B1,uuid:C54E838EEACFDA118C30CF8EA9BFE8B1
1,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,uuid:D9170E74C1CFDA118C30CF8EA9BFE8B1,uuid:A53BA05DE9CFDA118C30CF8EA9BFE8B1,uuid:18FD04FBB2CFDA118C30CF8EA9BFE8B1,uuid:B0B29CD2B8CFDA118C30CF8EA9BFE8B1
2,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,adobe:docid:photoshop:da668ae1-f595-11da-b021-...,None,None,None
3,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,adobe:docid:photoshop:da668ae7-f595-11da-b021-...,None,None,None
4,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,adobe:docid:photoshop:4e2d1ba0-f596-11da-b021-...,None,None,None
5,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,adobe:docid:photoshop:4e2d1ba6-f596-11da-b021-...,None,None,None
6,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,adobe:docid:photoshop:4e2d1baa-f596-11da-b021-...,None,None,None
7,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,adobe:docid:photoshop:93726e95-f596-11da-b021-...,None,None,None
8,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,adobe:docid:photoshop:eefb7ae0-f596-11da-b021-...,None,adobe:docid:photoshop:e7dc0224-3095-11d7-a2ec-...,uuid:e7dc0226-3095-11d7-a2ec-d1fcfb080b2c
9,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,adobe:docid:photoshop:eefb7ae6-f596-11da-b021-...,uuid:DEBD576D635FDA118C5D9BB46FE99D01,uuid:9EC36DC4625FDA118C5D9BB46FE99D01,uuid:9DC36DC4625FDA118C5D9BB46FE99D01



✔️ Dateien MIT XMP‑MM: 146 von 200


In [18]:
import re

def norm_id(raw: str|None) -> str|None:
    """
    Normalisiert Document/Instance-IDs:
    - trimmt Leerzeichen
    - belässt 'uuid:' und 'adobe:docid:photoshop:' Präfixe (wichtig für Eindeutigkeit)
    - wandelt in lowercase für stabile Vergleiche
    """
    if not raw: 
        return None
    s = raw.strip()
    return s.lower()

def extract_from_item(item: dict):
    """
    Holt IDs aus der JSON-Struktur (wie in deiner Beispielausgabe):
      - XMP-xmpMM:DocumentID
      - XMP-xmpMM:InstanceID
      - XMP-xmpMM:DerivedFrom{DocumentID,InstanceID}
    und normalisiert sie.
    """
    src = item.get("SourceFile")
    doc = item.get("XMP-xmpMM:DocumentID") or item.get("DocumentID")
    ins = item.get("XMP-xmpMM:InstanceID") or item.get("InstanceID")

    pdoc = pins = None
    drv = item.get("XMP-xmpMM:DerivedFrom") or item.get("DerivedFrom")
    if isinstance(drv, dict):
        pdoc = drv.get("DocumentID") or drv.get("documentID")
        pins = drv.get("InstanceID") or drv.get("instanceID")

    return {
        "file": src,
        "documentID": norm_id(doc),
        "instanceID": norm_id(ins),
        "parentDocumentID": norm_id(pdoc),
        "parentInstanceID": norm_id(pins),
    }

In [19]:
import os, json, subprocess, time, math
from pathlib import Path
import pandas as pd
import networkx as nx
from IPython.display import display, Markdown
from networkx.readwrite import json_graph

# ---------- Konfiguration ----------
ROOT = Path("/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein")
EXT = {".jpg",".jpeg",".tif",".tiff",".png",".psd",".psb"}
EXIFTOOL = "/home/renku/work/exiftool/exiftool"
SENTINEL = "__END__"
BATCH_SIZE = 100          # 50–200 sind oft gut. Bei Timeout: kleiner wählen.
FAST = False              # True → add -fast für Minimal‑I/O
OUT_PREFIX = "genealogy_full"

# ---------- Dateien einsammeln ----------
files = []
for r, d, flist in os.walk(ROOT):
    for f in flist:
        p = Path(r)/f
        if p.suffix.lower() in EXT:
            files.append(str(p))
total = len(files)
display(Markdown(f"**Zu scannen:** {total} Dateien"))

# ---------- ExifTool Stay‑Open starten ----------
args = [EXIFTOOL, "-stay_open", "True", "-@", "-"]
et = subprocess.Popen(args, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      text=True, bufsize=1)

def send_batch(batch_files):
    # Schlanke Argumente – nur xmpMM, optional -fast
    cmd = ["-a","-s","-struct","-XMP-xmpMM:all","-json"]
    if FAST:
        cmd.insert(0, "-fast")
    for a in cmd:
        et.stdin.write(a + "\n")
    for f in batch_files:
        et.stdin.write(f + "\n")
    # Sentinel für eindeutigen Abschluss
    et.stdin.write(f"-echo3\n{SENTINEL}\n")
    et.stdin.write("-execute\n")
    et.stdin.flush()

    # stdout bis Sentinel
    lines = []
    while True:
        line = et.stdout.readline()
        if not line:
            break
        if line.strip() == SENTINEL:
            break
        lines.append(line)

    out = "".join(lines).strip()
    if not out:
        return []

    # JSON Array isolieren
    i, j = out.find('['), out.rfind(']')
    if i == -1 or j == -1 or j < i:
        # Diagnose: kleine Vorschau
        print("⚠️ Kein JSON‑Array erkennbar. Vorschau:", out[:400])
        return []

    try:
        data = json.loads(out[i:j+1])
        return data if isinstance(data, list) else [data]
    except json.JSONDecodeError as e:
        print("⚠️ JSONDecodeError:", e)
        print("Vorschau:", out[max(0, i-100):min(len(out), j+100)])
        return []

def close_et():
    try:
        et.stdin.write("-stay_open\nFalse\n-execute\n")
        et.stdin.flush()
        et.communicate(timeout=3)
    except Exception:
        et.kill()

# ---------- Hauptlauf mit Fortschritt ----------
records = []
start = time.time()
processed = 0

try:
    for i in range(0, total, BATCH_SIZE):
        batch = files[i:i+BATCH_SIZE]
        t0 = time.time()
        data = send_batch(batch)
        for item in data:
            rec = extract_from_item(item)
            records.append(rec)
        processed += len(batch)
        # Fortschritt & ETA
        elapsed = time.time() - start
        rate = processed / elapsed if elapsed > 0 else 0
        remain = (total - processed) / rate if rate > 0 else 0
        print(f"\r{processed}/{total}  |  {rate:.2f} Dateien/s  |  ETA {remain/60:.1f} min", end="")
finally:
    close_et()

print()  # Zeilenumbruch

df = pd.DataFrame(records)
display(Markdown(f"**Mit XMP‑MM:** {(df['documentID'].notna()).sum() if 'documentID' in df.columns else 0} / {len(df)}"))

# ---------- Graph aufbauen ----------
G = nx.DiGraph()
for _, row in df.iterrows():
    doc, parent = row.get("documentID"), row.get("parentDocumentID")
    if doc: G.add_node(doc)
    if parent: G.add_edge(parent, doc)

display(Markdown(f"**Graph:** {G.number_of_nodes()} Knoten, {G.number_of_edges()} Kanten"))

# ---------- Exporte ----------
csv_path = f"{OUT_PREFIX}.csv"
json_graph_path = f"{OUT_PREFIX}_graph.json"
df.to_csv(csv_path, index=False)
with open(json_graph_path, "w") as f:
    json.dump(json_graph.node_link_data(G), f, indent=2)
print(f"CSV → {csv_path}")
print(f"Graph JSON → {json_graph_path}")

# ---------- Einfache Ketten-Ausgabe ----------
def print_chains(G, max_chains=5):
    roots = [n for n in G.nodes if G.in_degree(n) == 0]
    print(f"Gefundene Wurzeln: {len(roots)} (zeige bis zu {max_chains})")
    shown = 0
    for r in roots:
        if shown >= max_chains: break
        chain = list(nx.dfs_preorder_nodes(G, source=r))
        print(" → ".join(chain[:10]) + (" …" if len(chain) > 10 else ""))
        shown += 1

print_chains(G)

**Zu scannen:** 12338 Dateien

12338/12338  |  5.12 Dateien/s  |  ETA 0.0 min


**Mit XMP‑MM:** 6079 / 12338

**Graph:** 5992 Knoten, 59 Kanten

CSV → genealogy_full.csv
Graph JSON → genealogy_full_graph.json
Gefundene Wurzeln: 5933 (zeige bis zu 5)
uuid:c54e838eeacfda118c30cf8ea9bfe8b1 → uuid:c74e838eeacfda118c30cf8ea9bfe8b1
uuid:18fd04fbb2cfda118c30cf8ea9bfe8b1 → uuid:d9170e74c1cfda118c30cf8ea9bfe8b1
adobe:docid:photoshop:da668ae1-f595-11da-b021-8ba1b752bf42
adobe:docid:photoshop:da668ae7-f595-11da-b021-8ba1b752bf42
adobe:docid:photoshop:4e2d1ba0-f596-11da-b021-8ba1b752bf42


/home/renku/work/.venv/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


pfad zu graph hinzufügen

In [2]:
import json, pandas as pd
from collections import defaultdict

CSV = "genealogy_full.csv"
JSON_IN  = "genealogy_full_graph.json"
JSON_OUT = "genealogy_full_graph_with_paths.json"

# 1) CSV laden, Pfade je documentID sammeln
df = pd.read_csv(CSV)
paths_by_doc = defaultdict(list)
for _, row in df.iterrows():
    doc = str(row.get("documentID")).strip().lower() if pd.notna(row.get("documentID")) else None
    fp  = row.get("file")
    if doc:
        paths_by_doc[doc].append(fp)

# 2) JSON laden
graph = json.load(open(JSON_IN, "r"))

# 3) Nodes anreichern
for n in graph.get("nodes", []):
    doc = str(n.get("id")).strip().lower()
    files = sorted(set(paths_by_doc.get(doc, [])))
    n["files"] = files
    n["fileCount"] = len(files)
    n["fileSample"] = files[0] if files else None

# 4) Schreiben
with open(JSON_OUT, "w") as f:
    json.dump(graph, f, indent=2)

print("Angereichertes JSON:", JSON_OUT)

Angereichertes JSON: genealogy_full_graph_with_paths.json


In [2]:
import os, json, subprocess, time
from pathlib import Path
import pandas as pd
import networkx as nx
from IPython.display import display, Markdown
from networkx.readwrite import json_graph
from collections import defaultdict

# ---------- Konfiguration ----------
ROOT = Path("/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein")
EXT = {".jpg",".jpeg",".tif",".tiff",".png",".psd",".psb"}
EXIFTOOL = "/home/renku/work/exiftool/exiftool"
SENTINEL = "__END__"
BATCH_SIZE = 100          # 50–200 sind oft gut. Bei Timeout: kleiner wählen.
FAST = False              # True → add -fast für Minimal‑I/O
OUT_PREFIX = "genealogy_full"

# (NEU) Prioritäten für Klassen-Zusammenführung je Knoten
CLASS_PRIORITY = [
    "photoshop_derivative",
    "photoshop_no_parent",
    "camera_original",
    "bridge_or_raw",
    "unknown",
    "camera"  # nur für Kamera-Knoten selbst
]

# ---------- Dateien einsammeln ----------
files = []
for r, d, flist in os.walk(ROOT):
    for f in flist:
        p = Path(r) / f
        if p.suffix.lower() in EXT:
            files.append(str(p))
total = len(files)
display(Markdown(f"**Zu scannen:** {total} Dateien"))

# ---------- ExifTool Stay‑Open starten ----------
args = [EXIFTOOL, "-stay_open", "True", "-@", "-"]
et = subprocess.Popen(
    args,
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    bufsize=1
)

def send_batch(batch_files):
    """
    Ruft ExifTool im Stay-Open-Mode auf und liefert die JSON-Objekte für die Dateien.
    (NEU) Abfrage erweitert um EXIF/TIFF/CreatorTool, -G1 für stabile Feldnamen.
    """
    # Tags minimal & gezielt halten: XMP-xmpMM + EXIF/TIFF + CreatorTool
    cmd = [
        "-a", "-s", "-struct", "-G1", "-json",
        # MM-Genealogie
        "XMP-xmpMM:all",
        # Kamera-/EXIF-/TIFF
        "EXIF:Make", "EXIF:Model", "EXIF:DateTimeOriginal",
        "XMP-tiff:Make", "XMP-tiff:Model",
        # CreatorTool (Photoshop/Camera Raw/Bridge)
        "XMP-xmp:CreatorTool", "XMP-x:CreatorTool"
    ]
    if FAST:
        cmd.insert(0, "-fast")

    # Kommandos schreiben
    for a in cmd:
        et.stdin.write(a + "\n")
    for f in batch_files:
        et.stdin.write(f + "\n")
    # Sentinel für eindeutigen Abschluss
    et.stdin.write(f"-echo3\n{SENTINEL}\n")
    et.stdin.write("-execute\n")
    et.stdin.flush()

    # stdout bis Sentinel
    lines = []
    while True:
        line = et.stdout.readline()
        if not line:
            break
        if line.strip() == SENTINEL:
            break
        lines.append(line)

    out = "".join(lines).strip()
    if not out:
        return []

    # JSON Array isolieren
    i, j = out.find('['), out.rfind(']')
    if i == -1 or j == -1 or j < i:
        # Diagnose: kleine Vorschau
        print("⚠️ Kein JSON‑Array erkennbar. Vorschau:", out[:400])
        return []

    try:
        data = json.loads(out[i:j+1])
        return data if isinstance(data, list) else [data]
    except json.JSONDecodeError as e:
        print("⚠️ JSONDecodeError:", e)
        print("Vorschau:", out[max(0, i-100):min(len(out), j+100)])
        return []

def close_et():
    try:
        et.stdin.write("-stay_open\nFalse\n-execute\n")
        et.stdin.flush()
        et.communicate(timeout=3)
    except Exception:
        et.kill()

# ---------- (NEU) Extraktion & Klassifikation ----------
def extract_from_item(item):
    """
    Extrahiert XMP‑MM Genealogie + neue Klassifikation + Kamera-Infos.
    Erwartete Schlüssel (wegen -G1):
    - XMP-xmpMM:DocumentID / InstanceID / DerivedFrom:DocumentID / DerivedFrom:InstanceID
    - EXIF:Make/Model/DateTimeOriginal, XMP-tiff:Make/Model
    - XMP-xmp:CreatorTool oder XMP-x:CreatorTool
    """
    rec = {}

    # Basis
    rec["file"] = item.get("SourceFile")

    # MM-IDs
    rec["documentID"]       = item.get("XMP-xmpMM:DocumentID")
    rec["instanceID"]       = item.get("XMP-xmpMM:InstanceID")
    rec["parentDocumentID"] = item.get("XMP-xmpMM:DerivedFrom:DocumentID")
    rec["parentInstanceID"] = item.get("XMP-xmpMM:DerivedFrom:InstanceID")

    # Kamera
    make  = item.get("EXIF:Make") or item.get("XMP-tiff:Make")
    model = item.get("EXIF:Model") or item.get("XMP-tiff:Model")
    rec["cameraMake"] = make
    rec["cameraModel"] = model

    # CreatorTool
    creator = item.get("XMP-xmp:CreatorTool") or item.get("XMP-x:CreatorTool") or ""
    rec["creatorTool"] = creator

    # Flags
    has_parent = bool(rec["parentDocumentID"] or rec["parentInstanceID"])
    has_exif   = bool(make or model or item.get("EXIF:DateTimeOriginal"))

    is_photoshop = "photoshop" in creator.lower() if creator else False
    is_bridgeraw = (not is_photoshop) and has_exif  # Heuristik: XMP/EXIF ja, aber kein PS

    # Klasse
    if has_parent:
        rec["class"] = "photoshop_derivative"
    elif has_exif and (make or model):
        rec["class"] = "camera_original"
    elif is_photoshop and not has_parent:
        rec["class"] = "photoshop_no_parent"
    elif is_bridgeraw:
        rec["class"] = "bridge_or_raw"
    else:
        rec["class"] = "unknown"

    return rec

def pick_class(old_cls, new_cls):
    """Nimmt die Klasse mit höherer Priorität."""
    if not old_cls:
        return new_cls
    try:
        return old_cls if CLASS_PRIORITY.index(old_cls) <= CLASS_PRIORITY.index(new_cls) else new_cls
    except ValueError:
        return old_cls or new_cls

# ---------- Hauptlauf mit Fortschritt ----------
records = []
start = time.time()
processed = 0

try:
    for i in range(0, total, BATCH_SIZE):
        batch = files[i:i+BATCH_SIZE]
        t0 = time.time()
        data = send_batch(batch)
        for item in data:
            rec = extract_from_item(item)
            records.append(rec)
        processed += len(batch)
        # Fortschritt & ETA
        elapsed = time.time() - start
        rate = processed / elapsed if elapsed > 0 else 0
        remain = (total - processed) / rate if rate > 0 else 0
        print(f"\r{processed}/{total}  |  {rate:.2f} Dateien/s  |  ETA {remain/60:.1f} min", end="")
finally:
    close_et()

print()  # Zeilenumbruch

df = pd.DataFrame(records)
with_xmpmm = (df["documentID"].notna()).sum() if "documentID" in df.columns else 0
display(Markdown(f"**Mit XMP‑MM DocumentID:** {with_xmpmm} / {len(df)}"))

# ---------- Graph aufbauen (NEU: Klassen/Files/Kamera-Knoten) ----------
G = nx.DiGraph()

# Node-Aggregate
node_files  = defaultdict(set)
node_class  = defaultdict(str)

# (1) Zuerst alle Bild-Knoten erfassen (documentID oder fallback instanceID)
def get_node_id(row):
    return row["documentID"] if pd.notna(row.get("documentID")) else row.get("instanceID")

for _, row in df.iterrows():
    node_id = get_node_id(row)
    if not node_id:
        continue
    # Files sammeln
    fpath = row.get("file")
    if fpath:
        node_files[node_id].add(fpath)
    # Klasse zusammenführen
    node_class[node_id] = pick_class(node_class[node_id], row.get("class"))

# (2) Kamera-Knoten vorbereiten (nur wenn Kamera-Infos vorhanden)
def camera_id_from_row(row):
    mk, md = row.get("cameraMake"), row.get("cameraModel")
    if mk or md:
        # Lesbarer, aber stabiler Identifier:
        return f"camera:{(mk or '').strip()}|{(md or '').strip()}"
    return None

camera_nodes = set()
for _, row in df.iterrows():
    cam_id = camera_id_from_row(row)
    if cam_id:
        camera_nodes.add(cam_id)

# (3) Knoten in den Graph schreiben (mit Attributen class, files)
for nid in node_files.keys():
    G.add_node(nid, **{
        "class": node_class[nid] or "unknown",
        "files": sorted(node_files[nid])
    })

# (4) Kamera-Knoten hinzufügen (class=camera, files=[])
for cam in camera_nodes:
    if not G.has_node(cam):
        G.add_node(cam, **{"class": "camera", "files": []})

# (5) Genealogische Kanten (Parent → Child)
for _, row in df.iterrows():
    child = get_node_id(row)
    if not child:
        continue
    # parent via DocumentID / InstanceID
    pdoc = row.get("parentDocumentID")
    pinst = row.get("parentInstanceID")
    # Wir akzeptieren beide, erzeugen die Knoten bei Bedarf mit default-Attributen
    for parent in (pdoc, pinst):
        if parent and parent != child:
            if not G.has_node(parent):
                G.add_node(parent, **{"class": "unknown", "files": []})
            G.add_edge(parent, child)

# (6) Kamera → Bild-Kanten für camera_original
for _, row in df.iterrows():
    node_id = get_node_id(row)
    if not node_id:
        continue
    if row.get("class") == "camera_original":
        cam_id = camera_id_from_row(row)
        if cam_id:
            if not G.has_node(cam_id):
                G.add_node(cam_id, **{"class": "camera", "files": []})
            G.add_edge(cam_id, node_id)

display(Markdown(f"**Graph:** {G.number_of_nodes()} Knoten, {G.number_of_edges()} Kanten"))

# ---------- Exporte ----------
csv_path = f"{OUT_PREFIX}.csv"
json_graph_path = f"{OUT_PREFIX}_graph.json"
df.to_csv(csv_path, index=False)
with open(json_graph_path, "w") as f:
    json.dump(json_graph.node_link_data(G), f, indent=2, ensure_ascii=False)
print(f"CSV → {csv_path}")
print(f"Graph JSON → {json_graph_path}")

# ---------- Einfache Ketten-Ausgabe ----------
def print_chains(G, max_chains=5):
    roots = [n for n in G.nodes if G.in_degree(n) == 0]
    print(f"Gefundene Wurzeln: {len(roots)} (zeige bis zu {max_chains})")
    shown = 0
    for r in roots:
        if shown >= max_chains:
            break
        chain = list(nx.dfs_preorder_nodes(G, source=r))
        # Eine kurze, lesbare Darstellung:
        preview = " → ".join(chain[:10]) + (" …" if len(chain) > 10 else "")
        print(preview)
        shown += 1

print_chains(G)

**Zu scannen:** 12338 Dateien

12338/12338  |  5.09 Dateien/s  |  ETA 0.0 min


**Mit XMP‑MM DocumentID:** 6079 / 12338

**Graph:** 5971 Knoten, 65 Kanten

CSV → genealogy_full.csv
Graph JSON → genealogy_full_graph.json
Gefundene Wurzeln: 5906 (zeige bis zu 5)
uuid:C74E838EEACFDA118C30CF8EA9BFE8B1
uuid:D9170E74C1CFDA118C30CF8EA9BFE8B1
adobe:docid:photoshop:93726e95-f596-11da-b021-8ba1b752bf42
adobe:docid:photoshop:eefb7ae0-f596-11da-b021-8ba1b752bf42
adobe:docid:photoshop:17b71267-f597-11da-b021-8ba1b752bf42


/home/renku/work/.venv/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(
